In [26]:
import data_loader
import torch
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
from torchvision.transforms import v2 as transforms


## Dataset

In [27]:
# Defines the path to the folder with data in it
train_dir = Path("mbrimberry_files\Submissions")

In [ ]:
# Makes transforms and loads the datasets form the directories
trans = transforms.Compose(
    [
        transforms.ToImage(),
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Resize(size=(1056,816))
    ])
desk_data = data_loader.IndividualIMGDataset(targ_dir=train_dir,transform=trans)
packet_data = data_loader.IndividualIMGDataset(targ_dir=train_dir,transform=trans,type = "packet")
caddy_data = data_loader.IndividualIMGDataset(targ_dir=train_dir,transform=trans,type = "caddy")


In [ ]:
# The image is represented as a big tensor
# The labels and boxes are in a list which is returned second
packet_data[2]

## Displaying Dataset
These are a few methods for viewing the displayed data

Keep in mind that this is different then working with data_loader which stores the data in batches

In [ ]:
# You can use matplotlib to display the image tensor
import matplotlib.pyplot as plt
image, targets = packet_data[2]
print(f"{image.shape}")
image_permute = image.permute(1,2,0)
print(f"{image_permute.shape}")
fig, ax = plt.subplots(1)
ax.imshow(image_permute)

In [31]:
# This will return the image and target data for the third datapoint
img, targets = desk_data[2]

In [ ]:
# Image tensor
img

In [ ]:
# Coordinates for bounding boxes
# They are in center x, center y, width, heigh format
# The values go from 0.0 to 1.0 and go propotinal to the page (EX: 1.0 for x would mean center of the box is all the way to the right of the image)
targets["boxes"]

In [ ]:
# These labels are based on the class.txt file. 0 referes to the first entry and 1 refers to the second entry. 
targets["labels"]

In [ ]:
#Amount of desk images
len(desk_data)

In [ ]:
# Amount of packet images
len(packet_data)

In [ ]:
# These are the classes the correspond with 0 and 1 for the desk and packet data
desk_data.classes, packet_data.classes

In [ ]:
# This slider shows all of the images in the desk dataset with there bounding boxes
import ipywidgets as widgets
from IPython.display import display

# Display an image and its prediction
def display_prediction(index):
    image, targets = desk_data[index]
    bimg = data_loader.DrawBox(image,targets["boxes"],targets["labels"])
    image_permute = bimg.permute(1,2,0)
    plt.imshow(image_permute)
    plt.show()

# Slider widget (use the slider below)
slider = widgets.IntSlider(value=0, min=0, max=len(desk_data)-1, step=1, description='Index:')
widgets.interactive(display_prediction, index=slider)

In [ ]:
# This slider shows all of the images in the packet dataset with there bounding boxes
import ipywidgets as widgets
from IPython.display import display

# Display an image and its prediction
def display_prediction(index):
    image, targets = packet_data[index]
    bimg = data_loader.DrawBox(image,targets["boxes"],targets["labels"])
    image_permute = bimg.permute(1,2,0)
    plt.imshow(image_permute)
    plt.show()

# Slider widget (use the slider below)
slider = widgets.IntSlider(value=0, min=0, max=len(packet_data)-1, step=1, description='Index:')
widgets.interactive(display_prediction, index=slider)

## Dataloader

This section puts the dataset into the dataloader and we look at the data in a batch

In [40]:
BATCH_SIZE= 32
NUM_WORKERS = 0

dataloader = DataLoader(dataset=packet_data,
                        batch_size=BATCH_SIZE,
                        collate_fn=data_loader.collate_fn,
                        num_workers=NUM_WORKERS,
                        shuffle=False)

In [41]:
dataloader_desk = DataLoader(dataset=desk_data,
                        batch_size=BATCH_SIZE,
                        collate_fn=data_loader.collate_fn,
                        num_workers=NUM_WORKERS,
                        shuffle=False)

In [42]:
dataloader_caddy = DataLoader(dataset=caddy_data,
                        batch_size=BATCH_SIZE,
                        collate_fn=data_loader.collate_fn,
                        num_workers=NUM_WORKERS,
                        shuffle=False)

In [43]:
#To look at a batch you can use this
img, targets = next(iter(dataloader))

In [ ]:
# A for loop that goes through all the batches
# Might need to use this for CNN
# This tests all data for all batches
#
for imgs, targ in dataloader:
    print(imgs.shape, targ["boxes"].shape, targ["labels"].shape)

In [ ]:
targets["boxes"][0]

In [ ]:
img, targets = next(iter(dataloader))
from torchvision.utils import draw_bounding_boxes
bimg = data_loader.DrawBox(img[5],targets["boxes"][5],targets["labels"][5])
image_permute = bimg.permute(1,2,0)
fig, ax = plt.subplots(1)
ax.imshow(image_permute)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Display an image and its prediction
def display_prediction(index):
    image, targets = packet_data[index]
    bimg = data_loader.DrawBox(image,targets["boxes"],targets["labels"])
    image_permute = bimg.permute(1,2,0)
    plt.imshow(image_permute)
    plt.show()

# Slider widget (use the slider below)
slider = widgets.IntSlider(value=0, min=0, max=len(packet_data)-1, step=1, description='Index:')
widgets.interactive(display_prediction, index=slider)

## Displaying Dataloader
This makes the dataset and puts is through a dataloader.
It takes that dataloader extracts a batch from it and uses matplotlib to display it. 

In [ ]:
from pathlib import Path
import data_loader
import torch
from torchvision.transforms import v2 as transforms
trans = transforms.Compose(
    [
        transforms.ToImage(),
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Resize(size=(1056,816))
    ])
train_dir = Path("mbrimberry_files\Submissions")

#Can define type to "desk" or "packet" to change dataset
dataloader = data_loader.get_individual_data_loader(targ_dir=train_dir, transform = trans, type = "packet") 

# turns img and targ to be the batch of the next data
data = iter(dataloader)
img, targ = next(data)
    


In [ ]:
targ["labels"][0]

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Display an image and its prediction
def display_prediction(index):
    bimg = data_loader.DrawBox(img[index],targ["boxes"][index],targ["labels"][index])
    image_permute = bimg.permute(1,2,0)
    plt.imshow(image_permute)
    plt.show()

# Slider widget (use the slider below)
slider = widgets.IntSlider(value=0, min=0, max=len(img)-1, step=1, description='Index:')
widgets.interactive(display_prediction, index=slider)